# Transforming

In this notebook, we will explore how to use Large Language Models for text transformation tasks such as language translation, spelling and grammar checking, tone adjustment, and format conversion.

## Setup

In [1]:
import openai
import os

# from dotenv import load_dotenv, find_dotenv
# _ = load_dotenv(find_dotenv()) # read local .env file

# openai.api_key  = os.getenv('OPENAI_API_KEY')
openai.api_key  = ""

In [2]:
client = openai.AsyncOpenAI(
  api_key=openai.api_key,  # this is also the default, it can be omitted
)


async def get_completion(prompt_, model_="gpt-3.5-turbo"):
    messages_ = [{"role": "user", "content": prompt_}]
    response_ = await client.chat.completions.create(
        model=model_, 
        messages=messages_,
    )
    return response_.choices[0].message.content

## Translation

ChatGPT is trained with sources in many languages. This gives the model the ability to do translation. Here are some examples of how to use this capability.

In [3]:
prompt = f"""
Translate the following English text to Spanish: \ 
```Hi, I would like to order a blender```
"""
response = await get_completion(prompt)
print(response)

Hola, me gustaría ordenar una licuadora


In [4]:
prompt = f"""
Tell me which language this is: 
```Combien coûte le lampadaire?```
"""
response = await get_completion(prompt)
print(response)

French


In [5]:
prompt = f"""
Translate the following  text to French and Spanish
and English pirate: \
```I want to order a basketball```
"""
response = await get_completion(prompt)
print(response)

French: Je veux commander un ballon de basket
Spanish: Quiero ordenar un balón de baloncesto
English pirate:```I be wantin' to order a basketball```


In [6]:
prompt = f"""
Translate the following text to Spanish in both the \
formal and informal forms: 
'Would you like to order a pillow?'
"""
response = await get_completion(prompt)
print(response)

Formal: ¿Le gustaría ordenar una almohada?
Informal: ¿Te gustaría ordenar una almohada?


### Universal Translator
Imagine you are in charge of IT at a large multinational e-commerce company. Users are messaging you with IT issues in all their native languages. Your staff is from all over the world and speaks only their native languages. You need a universal translator!

In [7]:
user_messages = [
  "La performance du système est plus lente que d'habitude.",  # System performance is slower than normal         
  "Mi monitor tiene píxeles que no se iluminan.",              # My monitor has pixels that are not lighting
  "Il mio mouse non funziona",                                 # My mouse is not working
  "Mój klawisz Ctrl jest zepsuty",                             # My keyboard has a broken control key
  "我的屏幕在闪烁"                                               # My screen is flashing
] 

In [10]:
for issue in user_messages:
    prompt = f"Tell me what language this is: ```{issue}```"
    lang = await get_completion(prompt)
    print(f"Original message ({lang}): {issue}")

    prompt = f"""
    Translate the following  text to English \
    and Korean: ```{issue}```
    """
    response = await get_completion(prompt)
    print(response, "\n")

Original message (This is French.): La performance du système est plus lente que d'habitude.
English: "The system performance is slower than usual."

Korean: "시스템 성능이 평소보다 느립니다." 
Original message (This is Spanish.): Mi monitor tiene píxeles que no se iluminan.
English: My monitor has pixels that do not light up.

Korean: 제 모니터에는 빛나지 않는 픽셀이 있습니다. 
Original message (Italian): Il mio mouse non funziona
English: My mouse is not working
Korean: 내 마우스가 작동하지 않습니다 
Original message (Polish): Mój klawisz Ctrl jest zepsuty
English: My Ctrl key is broken.
Korean: 나의 Ctrl 키가 고장 났어요. 
Original message (This is Chinese.): 我的屏幕在闪烁
English: My screen is flickering

Korean: 내 화면이 깜박거린다 


## Try it yourself!
Try some translations on your own!

In [11]:
prompt = f"""
Tell me which language this is: 
```Великі собаки бігали лісом, а ні, то не собаки, то вовки.```
"""
response = await get_completion(prompt)
print(response)

Ukrainian


## Tone Transformation
Writing can vary based on the intended audience. ChatGPT can produce different tones.


In [12]:
prompt = f"""
Translate the following from slang to a business letter: 
'Dude, This is Joe, check out this spec on this standing lamp.'
"""
response = await get_completion(prompt)
print(response)

Dear Sir/Madam,

I am writing to introduce our new standing lamp product and provide you with the specifications. Thank you for your time and consideration.

Sincerely,
Joe


## Format Conversion
ChatGPT can translate between formats. The prompt should describe the input and output formats.

In [13]:
data_json = { "resturant employees" :[ 
    {"name":"Shyam", "email":"shyamjaiswal@gmail.com"},
    {"name":"Bob", "email":"bob32@gmail.com"},
    {"name":"Jai", "email":"jai87@gmail.com"}
]}

prompt = f"""
Translate the following python dictionary from JSON to an HTML \
table with column headers and title: {data_json}
"""
response = await get_completion(prompt)
print(response)

<html>
<head>
    <title>Restaurant Employees</title>
</head>
<body>
    <table border="1">
        <tr>
            <th>Name</th>
            <th>Email</th>
        </tr>
        <tr>
            <td>Shyam</td>
            <td>shyamjaiswal@gmail.com</td>
        </tr>
        <tr>
            <td>Bob</td>
            <td>bob32@gmail.com</td>
        </tr>
        <tr>
            <td>Jai</td>
            <td>jai87@gmail.com</td>
        </tr>
    </table>
</body>
</html>


In [14]:
from IPython.display import display, Markdown, Latex, HTML, JSON
display(HTML(response))

Name,Email
Shyam,shyamjaiswal@gmail.com
Bob,bob32@gmail.com
Jai,jai87@gmail.com


## Spellcheck/Grammar check.

Here are some examples of common grammar and spelling problems and the LLM's response. 

To signal to the LLM that you want it to proofread your text, you instruct the model to 'proofread' or 'proofread and correct'.

In [15]:
text = [ 
  "The girl with the black and white puppies have a ball.",  # The girl has a ball.
  "Yolanda has her notebook.", # ok
  "Its going to be a long day. Does the car need it’s oil changed?",  # Homonyms
  "Their goes my freedom. There going to bring they’re suitcases.",  # Homonyms
  "Your going to need you’re notebook.",  # Homonyms
  "That medicine effects my ability to sleep. Have you heard of the butterfly affect?", # Homonyms
  "This phrase is to cherck chatGPT for speling abilitty"  # spelling
]
for t in text:
    prompt = f"""Proofread and correct the following text
    and rewrite the corrected version. If you don't find
    and errors, just say "No errors found". Don't use 
    any punctuation around the text:
    ```{t}```"""
    response = await get_completion(prompt)
    print(response)

The girl with the black and white puppies has a ball.
No errors found.
No errors found.
Their goes my freedom. There going to bring they’re suitcases.

Errors found.

Corrected version:
There goes my freedom. They are going to bring their suitcases.
No errors found.
No errors found.
No errors found


In [22]:
text = f"""
Got this for my daughter for her birthday cuz she keeps taking \
mine from my room.  Yes, adults also like pandas too.  She takes \
it everywhere with her, and it's super soft and cute.  One of the \
ears is a bit lower than the other, and I don't think that was \
designed to be asymmetrical. It's a bit small for what I paid for it \
though. I think there might be other options that are bigger for \
the same price.  It arrived a day earlier than expected, so I got \
to play with it myself before I gave it to my daughter.
"""
prompt = f"proofread and correct grammar of this review: ```{text}```"
response = await get_completion(prompt)
print(response)

I got this for my daughter for her birthday because she keeps taking mine from my room. Yes, adults also like pandas too. She takes it everywhere with her, and it's super soft and cute. One of the ears is a bit lower than the other, and I don't think that was designed to be asymmetrical. It's a bit small for what I paid for it though. I think there might be other options that are bigger for the same price. It arrived a day earlier than expected, so I got to play with it myself before I gave it to my daughter.


In [23]:
from redlines import Redlines

diff = Redlines(text,response)
display(Markdown(diff.output_markdown))

<span style='color:red;font-weight:700;text-decoration:line-through;'>Got </span><span style='color:green;font-weight:700;'>I got </span>this for my daughter for her birthday <span style='color:red;font-weight:700;text-decoration:line-through;'>cuz </span><span style='color:green;font-weight:700;'>because </span>she keeps taking mine from my <span style='color:red;font-weight:700;text-decoration:line-through;'>room.  </span><span style='color:green;font-weight:700;'>room. </span>Yes, adults also like pandas <span style='color:red;font-weight:700;text-decoration:line-through;'>too.  </span><span style='color:green;font-weight:700;'>too. </span>She takes it everywhere with her, and it's super soft and <span style='color:red;font-weight:700;text-decoration:line-through;'>cute.  </span><span style='color:green;font-weight:700;'>cute. </span>One of the ears is a bit lower than the other, and I don't think that was designed to be asymmetrical. It's a bit small for what I paid for it though. I think there might be other options that are bigger for the same <span style='color:red;font-weight:700;text-decoration:line-through;'>price.  </span><span style='color:green;font-weight:700;'>price. </span>It arrived a day earlier than expected, so I got to play with it myself before I gave it to my daughter.

In [24]:
prompt = f"""
proofread and correct this review. Make it more compelling. 
Ensure it follows APA style guide and targets an advanced reader. 
Output in markdown format.
Text: ```{text}```
"""
response = await get_completion(prompt)
display(Markdown(response))

I purchased this adorable panda plush as a birthday gift for my daughter, as she constantly borrows mine from my room. It's not just for kids - even adults enjoy the charm of pandas. She adores carrying it everywhere, thanks to its irresistibly soft and cute design. However, I did notice a slight asymmetry in the ears, which seems unintentional. Also, considering the price, it is a bit smaller than I anticipated. Perhaps there are larger alternatives available at a similar cost. Despite these minor issues, the product arrived a day earlier than expected, giving me a chance to revel in its cuteness before presenting it to my daughter. Ultimately, while the size may not have been ideal, the early delivery and the overall appeal of this panda plush make it a delightful gift choice for anyone, regardless of age.

In [25]:
diff = Redlines(text,response)
display(Markdown(diff.output_markdown))

<span style='color:red;font-weight:700;text-decoration:line-through;'>Got </span><span style='color:green;font-weight:700;'>I purchased </span>this <span style='color:green;font-weight:700;'>adorable panda plush as a birthday gift </span>for my <span style='color:red;font-weight:700;text-decoration:line-through;'>daughter for her birthday cuz </span><span style='color:green;font-weight:700;'>daughter, as </span>she <span style='color:red;font-weight:700;text-decoration:line-through;'>keeps taking </span><span style='color:green;font-weight:700;'>constantly borrows </span>mine from my <span style='color:red;font-weight:700;text-decoration:line-through;'>room.  Yes, </span><span style='color:green;font-weight:700;'>room. It's not just for kids - even </span>adults <span style='color:red;font-weight:700;text-decoration:line-through;'>also like pandas too.  </span><span style='color:green;font-weight:700;'>enjoy the charm of pandas. </span>She <span style='color:red;font-weight:700;text-decoration:line-through;'>takes </span><span style='color:green;font-weight:700;'>adores carrying </span>it <span style='color:red;font-weight:700;text-decoration:line-through;'>everywhere with her, and it's super </span><span style='color:green;font-weight:700;'>everywhere, thanks to its irresistibly </span>soft and <span style='color:red;font-weight:700;text-decoration:line-through;'>cute.  One of </span><span style='color:green;font-weight:700;'>cute design. However, I did notice a slight asymmetry in </span>the <span style='color:red;font-weight:700;text-decoration:line-through;'>ears </span><span style='color:green;font-weight:700;'>ears, which seems unintentional. Also, considering the price, it </span>is a bit <span style='color:red;font-weight:700;text-decoration:line-through;'>lower </span><span style='color:green;font-weight:700;'>smaller </span>than <span style='color:green;font-weight:700;'>I anticipated. Perhaps there are larger alternatives available at a similar cost. Despite these minor issues, </span>the <span style='color:red;font-weight:700;text-decoration:line-through;'>other, and I don't think that was designed to be asymmetrical. It's a bit small for what I paid for it though. I think there might be other options that are bigger for the same price.  It </span><span style='color:green;font-weight:700;'>product </span>arrived a day earlier than expected, <span style='color:red;font-weight:700;text-decoration:line-through;'>so I got </span><span style='color:green;font-weight:700;'>giving me a chance </span>to <span style='color:red;font-weight:700;text-decoration:line-through;'>play with it myself </span><span style='color:green;font-weight:700;'>revel in its cuteness </span>before <span style='color:red;font-weight:700;text-decoration:line-through;'>I gave </span><span style='color:green;font-weight:700;'>presenting </span>it to my <span style='color:red;font-weight:700;text-decoration:line-through;'>daughter.</span><span style='color:green;font-weight:700;'>daughter. Ultimately, while the size may not have been ideal, the early delivery and the overall appeal of this panda plush make it a delightful gift choice for anyone, regardless of age.</span>

## Try it yourself!
Try changing the instructions to form your own review.

Thanks to the following sites:

https://writingprompts.com/bad-grammar-examples/